In [1]:
from pathlib import Path

def read_imdb_split(split_dir):
    split_dir = Path(split_dir)
    texts = []
    labels = []
    for label_dir in ["pos", "neg"]:
        for text_file in (split_dir/label_dir).iterdir():
            texts.append(text_file.read_text(encoding='utf-8'))
            labels.append(0 if label_dir == "neg" else 1)

    return texts, labels

train_texts, train_labels = read_imdb_split('aclImdb/train')
test_texts, test_labels = read_imdb_split('aclImdb/test')

In [2]:
from sklearn.model_selection import train_test_split
train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=.2)

In [3]:
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

In [4]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

In [5]:
import torch

class IMDbDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = IMDbDataset(train_encodings, train_labels)
val_dataset = IMDbDataset(val_encodings, val_labels)
test_dataset = IMDbDataset(test_encodings, test_labels)

In [8]:
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
)

model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset             # evaluation dataset
)

trainer.train()

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\Wen-Liang\AppData\Local\Programs\Python\Python312\Lib\site-packages\accelerate\accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


  0%|          | 0/3750 [00:00<?, ?it/s]

{'loss': 0.6898, 'grad_norm': 1.9999377727508545, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.01}
{'loss': 0.6916, 'grad_norm': 1.3765370845794678, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.02}
{'loss': 0.6917, 'grad_norm': 2.0320467948913574, 'learning_rate': 3e-06, 'epoch': 0.02}
{'loss': 0.6885, 'grad_norm': 0.9005399346351624, 'learning_rate': 4.000000000000001e-06, 'epoch': 0.03}
{'loss': 0.6967, 'grad_norm': 1.1405079364776611, 'learning_rate': 5e-06, 'epoch': 0.04}
{'loss': 0.6822, 'grad_norm': 1.042503833770752, 'learning_rate': 6e-06, 'epoch': 0.05}
{'loss': 0.6873, 'grad_norm': 1.2382616996765137, 'learning_rate': 7.000000000000001e-06, 'epoch': 0.06}
{'loss': 0.6719, 'grad_norm': 1.685051679611206, 'learning_rate': 8.000000000000001e-06, 'epoch': 0.06}
{'loss': 0.6308, 'grad_norm': 1.7533143758773804, 'learning_rate': 9e-06, 'epoch': 0.07}
{'loss': 0.5996, 'grad_norm': 2.601245403289795, 'learning_rate': 1e-05, 'epoch': 0.08}
{'loss': 0.4781, 'grad_norm': 6

TrainOutput(global_step=3750, training_loss=0.18540936918159326, metrics={'train_runtime': 2705.7738, 'train_samples_per_second': 22.175, 'train_steps_per_second': 1.386, 'train_loss': 0.18540936918159326, 'epoch': 3.0})